In [1]:
import sys
import os
#데이터용
from datetime import datetime
import numpy as np
import pandas as pd

#시각화용
import seaborn as sns
import matplotlib.pyplot as plt


In [2]:
from py_code.data_store import data_store
from py_code import config

There is data stored
Last data fetch completed


In [3]:
df_dict = data_store()

Load the saved pickle file


# Data Load

In [4]:
df = df_dict.return_dict_df()
df = df[df_dict.return_last_trial()]
trial = df_dict.return_last_trial()

In [18]:
train = df[df['data_set']=='train'].fillna(0)
test = df[df['data_set']=='test'].fillna(0)

# self,train,test,params = [],input_cols = [],

In [21]:
train.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,...,month,year,hour,dow,woy,count_season,peak,ideal,sticky,Td
datetime,,,,,,,,,,,,,,,,,,,,,
2011-01-01 00:00:00,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,...,1,2011,0,5,52,312498,0,0,0,6.731046
2011-01-01 01:00:00,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,...,1,2011,1,5,52,312498,0,0,0,5.751306
2011-01-01 02:00:00,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,...,1,2011,2,5,52,312498,0,0,0,5.751306
2011-01-01 03:00:00,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,...,1,2011,3,5,52,312498,0,0,0,5.614538
2011-01-01 04:00:00,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,...,1,2011,4,5,52,312498,0,0,0,5.614538


# Start EBM 

In [19]:
from interpret.glassbox import ExplainableBoostingRegressor

X_train = train[config.input_cols]
y_train = train['count']

ebm = ExplainableBoostingRegressor()
ebm.fit(X_train, y_train)

ExplainableBoostingRegressor(feature_names=['weather', 'temp', 'atemp',
                                            'windspeed', 'workingday', 'season',
                                            'holiday', 'hour', 'dow', 'woy',
                                            'peak', 'Td', 'workingday x hour',
                                            'hour x dow', 'woy x peak',
                                            'temp x hour', 'season x peak',
                                            'atemp x hour', 'season x hour',
                                            'hour x woy', 'atemp x workingday',
                                            'atemp x dow'],
                             feature_types=['continuous', 'continuous',
                                            'continuous', 'continuous',
                                            'categorical', 'continuous',
                                            'categorical', 'continuous',
                                    

# Explain 
> Dist  
> Global  
> Local  

In [57]:
from interpret import show
from interpret.data import Marginal

marginal = Marginal().explain_data(X_train, y_train, name = 'Train Data')
show(marginal)

<!-- http://127.0.0.1:7001/140363484042896/ -->

In [22]:
ebm_global = ebm.explain_global()
show(ebm_global)

/Users/youngyong/opt/anaconda3/lib/python3.7/site-packages/interpret/visual/udash.py:5: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
/Users/youngyong/opt/anaconda3/lib/python3.7/site-packages/interpret/visual/udash.py:6: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
/Users/youngyong/opt/anaconda3/lib/python3.7/site-packages/interpret/visual/udash.py:7: UserWarning: 
The dash_table package is deprecated. Please replace
`import dash_table` with `from dash import dash_table`

Also, if you're using any of the table format helpers (e.g. Group), replace 
`from dash_table.Format import Group` with 
`from dash.dash_table.Format import Group`
  import dash_table as dt


<!-- http://127.0.0.1:7001/140363449251792/ -->

- EBM을 이용한 EDA를 해보자!
> 1. 각 column의 가장 score가 높은점과 낮은점을 구하자
> 2. 각 column의 기울기가 변하는 구간을 구하자
> 3. 


In [216]:
cols = config.input_cols

def get_eachCols_EBM_score(ebm_global, cols):
    EBM_cols_dict = dict()
    for i in range(len(cols)):
        EBM_cols_dict[cols[i]] = {'values':ebm_global.data(i)['names'],'scores':list(ebm_global.data(i)['scores'])}
    return EBM_cols_dict
EBM_cols_dict = get_eachCols_EBM_score(ebm_global, cols)
EBM_cols_dict['weather']

{'values': [1.0, 1.5, 2.5, 3.5, 4.0],
 'scores': [5.9336914049573295,
  -0.10373869749127351,
  -49.280364088096306,
  -49.280364088096306]}

In [215]:
def get_EBM_MinMax_scorePoint(df, EBM_cols_dict):
    EBM_Score_Min_max_df = dict()
    for k,v in EBM_cols_dict.items():
        EBM_Score_Min_max_df[k] = {'Unique':len(set(df[k])),'Max_Point':v['values'][v['scores'].index(max(v['scores']))],'Min_Point':v['values'][v['scores'].index(min(v['scores']))] ,'Max_Score' : int(max(v['scores'])), 'Min_Score' : int(min(v['scores'])), 'Midean' : int(np.median(v['scores']))}
    EBM_Score_Min_max_df = pd.DataFrame(EBM_Score_Min_max_df).transpose()
    EBM_Score_Min_max_df['Range_Score'] = EBM_Score_Min_max_df['Max_Score'] - EBM_Score_Min_max_df['Min_Score']
    return EBM_Score_Min_max_df.sort_values('Range_Score',ascending=False)
get_EBM_MinMax_scorePoint(train,EBM_cols_dict)


,Unique,Max_Point,Min_Point,Max_Score,Min_Score,Midean,Range_Score
peak,2,1.0,0.0,254,-24,115,278
hour,24,18.5,3.5,131,-134,27,265
Td,969,3.77055,25.1054,63,-112,9,175
temp,49,30.75,9.43,100,-69,-6,169
atemp,60,39.7725,13.2575,96,-41,-9,137
woy,46,37.5,51.5,53,-77,-1,130
windspeed,28,19.5004,39.999,5,-94,-5,99
weather,4,1,2.5,5,-49,-24,54
workingday,2,0.0,1.0,27,-12,7,39
season,4,3.5,1,13,-8,-2,21


In [212]:
train['workingday'].value_counts()

1    7460
0    3426
Name: workingday, dtype: int64

In [131]:
from scipy.stats import linregress
x = ebm_global.data(0)['names'][1:]
y = ebm_global.data(0)['scores']

# slope, intercept, r_value, p_value, std_err = linregress(x, y)
# print(slope)


import numpy as np

slope, intercept = np.polyfit(x,y,1)
print(slope)

-25.589304630352725


In [39]:
ebm_local = ebm.explain_local(X_train, y_train)
show(ebm_local)

<!-- http://127.0.0.1:7001/140363468588688/ -->

# 이건 뭐지

In [42]:
from interpret.privacy import DPExplainableBoostingClassifier, DPExplainableBoostingRegressor

dp_ebm = DPExplainableBoostingRegressor(epsilon=1, delta=1e-5) # Specify privacy parameters
dp_ebm.fit(X_train, y_train)

show(dp_ebm.explain_global()) # Identical function calls to standard EBMs

<!-- http://127.0.0.1:7001/140363480820560/ -->